## Trabajo Final Grupal

### Objetivo: Encontrar los rubros que más crecen y decrecen a través de un análisis sentiment de los comentarios. 

In [5]:
# Importar librería:
import pandas as pd
import numpy as np
import json
import ast
#from textblob import TextBlob
from pandas import json_normalize
import seaborn as sns
import matplotlib.pyplot as plt
#import plotly.express as px
from wordcloud import WordCloud
#import plotly.graph_objs as go
#from sentiment_analysis_spanish import sentiment_analysis
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

### Análisis Exploratorio: 

- Los comentarios de Yelp están en la tabla 'reviews'. Las tablas de Yelp están conectadas por el 'business_id'
- Los comentarios de Gmaps están en gmaps del estado de Florida.


In [2]:
data_gmaps = pd.read_csv("gmaps_FL.csv", delimiter = ",")
data_gmaps.head(2)
#data_2.columns

,user_id,name,time,rating,text,pics,resp,gmap_id
0,1.014719e+20,Julie A. Gerber,1628003250740,1,Update: Their “reply” to my review amounted to...,NaN,"{'time': 1627042799532, 'text': 'Thank you for...",0x8893863ea87bd5dd:0x9383ebf973e74abb
1,1.154772e+20,Martin Sheffield,1595031217005,5,He's a knowledgeable doctor but the way he run...,NaN,"{'time': 1582464056733, 'text': 'Thank you for...",0x8893863ea87bd5dd:0x9383ebf973e74abb


In [3]:
data_gmaps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2850000 entries, 0 to 2849999
Data columns (total 8 columns):
 #   Column   Dtype  
---  ------   -----  
 0   user_id  float64
 1   name     object 
 2   time     int64  
 3   rating   int64  
 4   text     object 
 5   pics     object 
 6   resp     object 
 7   gmap_id  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 174.0+ MB


In [4]:
# Totales por columnas
data_gmaps.nunique()

user_id     903683
name        781029
time       2729455
rating           5
text       1470127
pics         95810
resp        436417
gmap_id      59200
dtype: int64

In [5]:
data_gmaps.isnull().sum()

user_id          0
name            17
time             0
rating           0
text       1079547
pics       2745810
resp       2394601
gmap_id          0
dtype: int64

In [3]:
# Eliminar filas con valores nulos de la columna 'id': df.dropna(subset=['id'])
# Eliminar columnas que no usaremos
data_gmaps = data_gmaps.drop(['pics', 'resp'], axis=1)
data_gmaps.columns

Index(['user_id', 'name', 'time', 'rating', 'text', 'gmap_id'], dtype='object')

In [8]:
# Convierte los elementos de tipo float a cadenas en la columna 'text'
data_gmaps['text'] = data_gmaps['text'].apply(lambda x: str(x) if isinstance(x, float) else x)

In [15]:
#Se analizan los distintos valores de la columna Operador para identificar posibles análisis
data_gmaps.rating.value_counts()

rating
5    1870990
4     476149
3     221512
1     192856
2      88493
Name: count, dtype: int64

In [4]:
data_yelp = pd.read_csv("business_yelp_florida.csv", delimiter = ",")
data_yelp.head(2)
#data_business

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,UJsufbvfyfONHeWdvAHKjA,Marshalls,21705 Village Lakes Sc Dr,Land O' Lakes,FL,34639,28.190459,-82.457380,3.5,6,1,"{'RestaurantsPriceRange2': '2', 'BikeParking':...","Department Stores, Shopping, Fashion","{'Monday': '9:30-21:30', 'Tuesday': '9:30-21:3..."
1,jaxMSoInw8Poo3XeMJt8lQ,Adams Dental,15 N Missouri Ave,Clearwater,FL,33755,27.966235,-82.787412,5.0,10,1,{'ByAppointmentOnly': 'True'},"General Dentistry, Dentists, Health & Medical,...","{'Monday': '7:30-15:30', 'Tuesday': '7:30-15:3..."


In [5]:
# Verificación del tipo de dato de la columna 'fecha':
data_yelp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26329 entries, 0 to 26328
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   26329 non-null  object 
 1   name          26329 non-null  object 
 2   address       25413 non-null  object 
 3   city          26329 non-null  object 
 4   state         26329 non-null  object 
 5   postal_code   26321 non-null  object 
 6   latitude      26329 non-null  float64
 7   longitude     26329 non-null  float64
 8   stars         26329 non-null  float64
 9   review_count  26329 non-null  int64  
 10  is_open       26329 non-null  int64  
 11  attributes    23883 non-null  object 
 12  categories    26311 non-null  object 
 13  hours         22221 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 2.8+ MB


In [6]:
data_yelp.state.value_counts()

state
FL    26329
Name: count, dtype: int64

In [7]:
#Se analizan los distintos valores de la columna Operador para identificar posibles análisis
data_yelp.categories.value_counts() 

categories
Beauty & Spas, Nail Salons                                                   176
Restaurants, Pizza                                                           163
Nail Salons, Beauty & Spas                                                   153
Pizza, Restaurants                                                           142
Mexican, Restaurants                                                         122
                                                                            ... 
Vegan, Gluten-Free, Restaurants                                                1
Nail Salons, Hair Removal, Beauty & Spas, Nail Technicians, Waxing             1
Financial Services, Tax Services                                               1
Interior Design, Home & Garden, Furniture Stores, Home Services, Shopping      1
Retirement Homes, Health & Medical, Hospice, Rehabilitation Center             1
Name: count, Length: 17457, dtype: int64

In [8]:
with open('review.json', 'r') as f:
        review = [json.loads(line) for line in f]

data_review = pd.DataFrame(review)
data_review.head(2)

In [ ]:
data_review_florida = data_review[data_review['business_id'].isin(data_yelp['business_id'])]
data_review_florida.head(2)

In [ ]:
# Eliminar la columnas que no usaremos
df2 = df1.drop(columns=['genres', 'title', 'specs', 'price', 'early_access', 'metascore', 'year', 'playtime_forever'])
df2.head(2)

In [5]:
# Transformacion de Tags a columnas 
# Llenamos los valores de las columnas con datos nulos
df_steam_games['tags'] = df_steam_games['tags'].fillna('[]')
df_steam_games['tags'] = df_steam_games['tags'].apply(lambda x: ast.literal_eval(str(x)))
data_games_tags = pd.DataFrame(df_steam_games['tags'].tolist())
data_games_tags_obj = data_games_tags.stack()
data_games_tags_df = pd.get_dummies(data_games_tags_obj)

In [6]:
data_games_tags_df = data_games_tags_df.groupby(level=[0],axis=0).sum()
data_games_tags_df.head(2)

,1980s,1990's,2.5D,2D,2D Fighter,360 Video,3D Platformer,3D Vision,4 Player Local,4X,...,Warhammer 40K,Web Publishing,Werewolves,Western,Word Game,World War I,World War II,Wrestling,Zombies,e-sports
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
data_games_tags_df.sum().sort_values(ascending=False).head(20)/len(data_games_tags_df)

Indie               0.551124
Action              0.406744
Adventure           0.307466
Casual              0.304620
Simulation          0.241782
Strategy            0.236652
RPG                 0.185137
Singleplayer        0.135904
Free to Play        0.075256
Multiplayer         0.074536
Great Soundtrack    0.069907
Puzzle              0.066029
Early Access        0.060899
2D                  0.060836
Atmospheric         0.060492
VR                  0.056176
Sports              0.049952
Platformer          0.045322
Story Rich          0.045197
Sci-fi              0.043414
dtype: float64

In [8]:
temp_df = data_games_tags_df[['Action','Indie','Adventure','Casual','Fighting','Free to Play','Multiplayer',
    'Puzzle','RPG','Sandbox','Shooter','Simulation','Singleplayer','Sports','Strategy','Survival','Zombies']]
temp_df.head(2)

,Action,Indie,Adventure,Casual,Fighting,Free to Play,Multiplayer,Puzzle,RPG,Sandbox,Shooter,Simulation,Singleplayer,Sports,Strategy,Survival,Zombies
0,1,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0
1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0


In [9]:
df_steam_games = pd.concat([df_steam_games,temp_df],axis=1,ignore_index=True)
df_steam_games.head(2)

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,4.49,[Single-player],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,NaN,"[Single-player, Multi-player, Online Multi-Pla...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Grabar el dataframe data_geres, specs sen un archivo CSV
data_geres.to_csv('data_geres.csv', header=True, index=False)
data_specs.to_csv('data_specs.csv', header=True, index=False)
data_tags.to_csv('data_tags.csv', header=True, index=False)

# import pandas as pd
df_geres = pd.read_csv('data_geres.csv')
df_geres = df_geres.rename(columns={'Unnamed: 0': 'NuevoNombre'})
df_geres = df_geres.drop(['NuevoNombre'], axis=1)
df_geres.head(2)

### Sentiments
En el dataset user_reviews se incluyen reseñas de juegos hechos por distintos usuarios. Debes crear la columna 'sentiment_analysis' aplicando análisis de sentimiento con NLP con la siguiente escala: debe tomar el valor '0' si es malo, '1' si es neutral y '2' si es positivo. Esta nueva columna debe reemplazar la de user_reviews.review para facilitar el trabajo de los modelos de machine learning y el análisis de datos. De no ser posible este análisis por estar ausente la reseña escrita, debe tomar el valor de 1.

def sentiment_analysis(año : int): Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento. Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}

Nuestro cliente es parte de un conglomerado de empresas de restaurantes y afines, y desean tener un análisis detallado de la opinión de los usuarios en Yelp y cruzarlos con los de Google Maps sobre hoteles, restaurantes y otros negocios afines al turismo y ocio, utilizando análisis de sentimientos.
- Predecir cuáles serán los rubros de los negocios que más crecerán (o decaerán). 
- Dónde es conveniente reemplazar los nuevos locales de restaurantes y afines. 

Sistema de recomendación de restaurantes para los usuarios de ambas plataformas para darle, al usuario por ejemplo la posibilidad de poder conocer nuevos sabores basados en sus experiencias previas.

In [1]:
import pandas as pd

In [3]:
# Lectura de datos del archivo ETL_business 
df_review_categories = pd.read_csv('review_cities_categories.csv')
df_review_categories.shape

(249056, 34)

In [4]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\n_sar\AppData\Roaming\nltk_data...


True

In [5]:
def analizar_sentimiento(texto):
    sia = SentimentIntensityAnalyzer()
    polaridad = sia.polarity_scores(texto)['compound']
    
    if polaridad > 0.05:
        return 'Positiva'
    elif polaridad < -0.05:
        return 'Negativa'
    else:
        return 'Neutra'

In [ ]:
df_review_categories['sentiment_analysis'] = df_review_categories['text'].apply(analizar_sentimiento)   
df_review_categories.head(2)

### Modelo de aprendizaje automático
Una vez que toda la data es consumible por la API, está lista para consumir por los departamentos de Analytics y Machine Learning, y nuestro EDA nos permite entender bien los datos a los que tenemos acceso, es hora de entrenar nuestro modelo de machine learning para armar un sistema de recomendación. 

Para ello, te ofrecen dos propuestas de trabajo: 
En la primera, el modelo deberá tener una relación ítem-ítem, esto es se toma un item, en base a que tan similar esa ese ítem al resto, se recomiendan similares. Aquí el input es un juego y el output es una lista de juegos recomendados, para ello recomendamos aplicar la similitud del coseno. 

In [2]:
df1 = pd.read_csv('data_1000.csv')
df1.head(2)

,name_gmaps,address_gmaps,gmap_id,avg_rating_gmaps,business_id,name_yelp,address_yelp,city,stars_yelp,user_id_gmaps,text_gmaps,review_id_yelp,user_id,text_yelp,review_count_user_yelp,average_stars
0,KUBA EN LA PLAYA,"KUBA EN LA PLAYA, 7525 Blind Pass Rd, St Pete ...",0x88c2fdf85af1beab:0xe3181616d85e0c32,4.7,IjtKwA7njLjTvZr0nFSLeA,Kuba,7525 Blind Pass Rd,St. Pete Beach,4.0,1.151756e+20,Hands down the best Cuban I've had. A great sp...,RxiPnO9K2-Wh4ZwjDA97Pg,8OxNMS2n6PuKip-1oHVweQ,Good food but the quality doesn't seem to just...,7,2.73
1,KUBA EN LA PLAYA,"KUBA EN LA PLAYA, 7525 Blind Pass Rd, St Pete ...",0x88c2fdf85af1beab:0xe3181616d85e0c32,4.7,IjtKwA7njLjTvZr0nFSLeA,Kuba,7525 Blind Pass Rd,St. Pete Beach,4.0,1.151027e+20,Stopped by there for lunch.\nGot my wife the e...,RxiPnO9K2-Wh4ZwjDA97Pg,8OxNMS2n6PuKip-1oHVweQ,Good food but the quality doesn't seem to just...,7,2.73


In [13]:
# Eliminar la columna 'release_date'
df2 = df1.drop(columns=['genres', 'title', 'specs', 'price', 'early_access', 'metascore', 'year', 'playtime_forever'])
df2.head(2)

,id,Action,Casual,Indie,Simulation,Strategy,Free to Play,RPG,Sports,Adventure,...,Video Production,Utilities,Web Publishing,Education,Software Training,Design &amp; Illustration,Audio Production,Photo Editing,Accounting,recommend
0,282010,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,70,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61


In [45]:

# Reemplaza 'df_gmap_state' con el nombre real de tu DataFrame
df = pd.read_csv('florida_reviews.csv')
df_3 = df.head(30)

# Paso 1: Calcular la matriz TF-IDF de los textos
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df_3['text'].values.astype('U'))

# Paso 2: Calcular similitudes de coseno entre los textos
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Paso 3: Crear un mapeo entre los nombres de negocios y sus índices
indices = pd.Series(df_3.index, index=df_3['name']).drop_duplicates()
indices
# Función para obtener recomendaciones
def obtener_recomendaciones(nombre_negocio, cosine_sim=cosine_sim):
    # Obtenemos el índice del negocio según su nombre
    if nombre_negocio in indices:
        idx = indices[nombre_negocio]
    else:
        return ["No se encontro el negocio"]
    # Obtenemos las puntuaciones de similitud de coseno para todos los negocios
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Ordenamos los negocios según las puntuaciones de similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Obtenemos los índices de los 5 negocios más similares
    top_indices = [i[0] for i in sim_scores[1:6]]

    # Devolvemos los nombres de los negocios más similares
    return df_3['name'].iloc[top_indices]



In [47]:
# Ejemplo de uso:
nombre_negocio = "David Delgado"
recomendaciones = obtener_recomendaciones(nombre_negocio)

print(f"Recomendaciones para '{nombre_negocio}':")
for nombre in recomendaciones:
    print(nombre)

Recomendaciones para 'David Delgado':
Douglas Gummere
David Powell
Hillary Hutchins
Ajay patel
Desiree Martina
